In [1]:
from utilities_IGNOBEL import *
with open('credential.json','r') as f:
    cred = json.load(f)
cluster = MongoClient(cred['cred'])
db = cluster['Game']
collection = db['Players']
import glob 
import re

In [4]:
"""Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato"""
#RoseSS = rose_complete()

#RoseFG = rose()

#labels = RoseFG.columns
#Rose = RoseSS[labels]

'Usare queste rose se FG non ha quelle corrette, e.g. durante il mercato'

In [4]:
#infortunati(giornata=1) #restituisce una singola lista con tutti gli infortunati di ogni squadra di serie A per la giornata indicata

In [2]:
#rose() #restituisce le rose attuali di ogni fantallenatore

In [3]:
#formazioni() #restituisce le formazioni schierate in ogni giornata (inclusa la panchina)

In [2]:
#per qualche strano motivo a volte da errore la prima volta che gira, 
#ma funziona le volte successive, potrebbe essere un caso
V = bonus_panchina(giornata=1) 
V
# restituisce i voti dei panchinari che non sono scesi in campo (link diverso temp)

,MAINZ NA GIOIA,I DISEREDATI,PDG 1908,SOROS FC,XYZ,AS 800A,PALLA PAZZA,IGNORANZA EVERYWHERE
Bonus Panchinari,0,2.0,3.75,0.5,1.5,4.25,1.5,0


In [3]:
G = goal_subiti(giornata=1)
G
# restituisce il totale dei gol subiti da ogni portiere sceso in campo

,MAINZ NA GIOIA,I DISEREDATI,PDG 1908,SOROS FC,XYZ,AS 800A,PALLA PAZZA,IGNORANZA EVERYWHERE
Goal subiti,1,2,1,0,0,0,4,2


In [4]:
M = modificatore(giornata=1) 
M
# restituisce il bonus modificatore difesa guadagnato da ogni squadra

,MAINZ NA GIOIA,I DISEREDATI,PDG 1908,SOROS FC,XYZ,AS 800A,PALLA PAZZA,IGNORANZA EVERYWHERE
Modificatore,0,0,0,1.0,0,0,1.0,0


In [5]:
C = cartellini(giornata=1)
C

,MAINZ NA GIOIA,I DISEREDATI,PDG 1908,SOROS FC,XYZ,AS 800A,PALLA PAZZA,IGNORANZA EVERYWHERE
C. gialli,1,1,1,1,2,4,0,2
C. rossi,0,1,1,0,2,0,0,0


In [4]:
CI = count_inf(infortunati(giornata = 3), rose()) #restituisce il numero di infortunati in ogni rosa
##NOTA: usa Rose definito sopra nel caso in cui FG non ha le rose corrette e quindi serve usare lo spreadsheet
CI

,XYZ,MAINZ NA GIOIA,I DISEREDATI,PALLA PAZZA,AS 800A,PDG 1908,SOROS FC,IGNORANZA EVERYWHERE
tot Infortunati,3,1,0,3,2,3,2,3


In [7]:
F = fantapunti_fatti(giornata=1)

In [41]:
S = fantapunti_subiti(giornata=21)

In [42]:
#tot = IGNOBEL_tot(giornata=6) # a causa di ritardi di connessione, potrebbe dare errore le prime volte: in tal caso far girare nuovamente
output = pd.concat([F,S,G,C,V,M,CI], axis = 0).T #enormi problemi di connessione, quindi ho dovuto dividere in singole tasks
tot = output.T
tot

,IGNORANZA EVERYWHERE,MAINZ NA GIOIA,XYZ,AS 800A,PALLA PAZZA,I DISEREDATI,SOROS FC,PDG 1908
Fantapunti Fatti,64.0,69.0,64.00,71.25,72.75,75.50,67.50,72.75
Fantapunti Subiti,69.0,64.0,71.25,64.00,75.50,72.75,72.75,67.50
Goal subiti,1.0,2.0,2.00,0.00,0.00,0.00,0.00,0.00
C. gialli,0.0,3.0,2.00,1.00,1.00,0.00,0.00,0.00
C. rossi,1.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00
Bonus Panchinari,0.5,0.0,0.00,1.50,0.00,1.50,0.00,0.00
Modificatore,0.0,0.5,0.50,0.00,1.50,1.50,0.00,1.50
tot Infortunati,4.0,4.0,1.00,4.00,1.00,4.00,2.00,5.00


In [8]:
#G = pd.read_pickle("Dati_storici/Giornata_22.pkl")
#G

In [5]:
def df_to_dict(df):
    as_list=df.index.tolist()
    idxG = as_list.index('C. gialli')
    idxR = as_list.index('C. rossi')
    as_list[idxG] = 'C_gialli'
    as_list[idxR] = 'C_rossi'
    df.index=as_list
    temp = dict(df)
    temp_2 = {}
    for key, element in temp.items():
        temp_2[key] = dict(element)
    return temp_2

In [6]:
def get_matchday_dict(k):
    files = glob.glob('Dati_storici/*.pkl')
    dict_matchday={}
    for file in files:
        num = re.findall('_\d+.',file)[0][1:-1] 
        dict_matchday[int(num)]=file
    df=pd.read_pickle(dict_matchday[k])
    return df_to_dict(df)

In [7]:
def create_results_post(season, matchday, df, post_to_mongo=False):
    idx=season+'_G'+f'{matchday:02}'
    post = {'_id': idx,
     'matchday':matchday,
     'season':season,
     'results': df_to_dict(df)
    }
    if post_to_mongo:
        coll_res = db['Results']
        if len(list(coll_res.find({'_id': idx})))>0:
            coll_res.delete_one({'_id': idx})
        coll_res.insert_one(post)
        print('Results added to MongoDB')
    else:    
        return post

In [12]:
giornata=3
while True:
    try:
        tot = IGNOBEL_tot(giornata)
        break
    except:
        print('retrying')
        continue
        
create_results_post('2021-22', giornata, tot, post_to_mongo=True)
tot


#tot.to_pickle("Dati_storici/Giornata_"+str(giornata)+".pkl")
#crea 8 dataframe individuali con i dati aggiornati fino alla giornata indicata
#aggiorna_database(giornata)

Results added to MongoDB


,PDG 1908,IGNORANZA EVERYWHERE,XYZ,I DISEREDATI,PALLA PAZZA,MAINZ NA GIOIA,AS 800A,SOROS FC
Fantapunti Fatti,72.75,69.00,67.5,69.0,65.50,70.25,66.25,71.00
Fantapunti Subiti,69.00,72.75,69.0,67.5,70.25,65.50,71.00,66.25
Goal subiti,2.00,0.00,0.0,3.0,2.00,1.00,2.00,1.00
C_gialli,1.00,1.00,0.0,0.0,4.00,2.00,2.00,0.00
C_rossi,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.00
Bonus Panchinari,0.00,0.00,3.0,0.0,2.75,5.50,0.00,2.00
Modificatore,1.00,1.00,0.0,1.5,0.50,0.00,0.00,0.00
tot Infortunati,3.00,3.00,3.0,0.0,3.00,1.00,2.00,2.00


In [5]:
#voti_all = scarica_voti(giornata = 3, stagione = '2020-21')
#scarica il dataframe con tutti i voti direttamente da fantacalcio.it

In [7]:
storico_individuale(nome = 'luca', giornata = 1) 
#crea un dataframe individuale con le informazioni utili per gli IGNOBEL, con i dati fino alla giornata scelta
#i dati vengono pescati dai pkl contenuti nella cartella Dati_storici

,gg,pf,ps,gs,c,pan,mod,inf,nome
1,1,72.25,70.5,0,6,1.5,0,0,luca


In [2]:
#include svincolati
#ST = stats_by_team_full(primavera = True)
ST_INFO = stats_by_team_NO_INFO(stagione = '2021-22',primavera = True) #NO PERSONAL DATA
missing_pl = info_missing_players(collection, ST_INFO) #missing players from our db

100% (33 of 33) |########################| Elapsed Time: 0:00:42 Time:  0:00:42


In [40]:
"""database=ST_INFO
missing_ids = find_missing_players(collection, database)
all_pl = []
k=0
while k < len(missing_ids):
    idx = missing_ids[k]
    try:
        infos = personal_info(idx, database)
    except:
        k = k-1
        continue
    dic = {}
    dic['Id'] = idx
    dic['Classe'] = infos[0]
    dic['Eta\''] = infos[1]
    dic['Nome Completo'] = infos[2]
    dic['Nazionalita\''] = infos[3]
    all_pl.append(dic)
    print(k/len(missing_ids))
    k+=1
missing_pl=pd.DataFrame(all_pl, index = missing_ids)"""

"database=ST_INFO\nmissing_ids = find_missing_players(collection, database)\nall_pl = []\nk=0\nwhile k < len(missing_ids):\n    idx = missing_ids[k]\n    try:\n        infos = personal_info(idx, database)\n    except:\n        k = k-1\n        continue\n    dic = {}\n    dic['Id'] = idx\n    dic['Classe'] = infos[0]\n    dic['Eta''] = infos[1]\n    dic['Nome Completo'] = infos[2]\n    dic['Nazionalita''] = infos[3]\n    all_pl.append(dic)\n    print(k/len(missing_ids))\n    k+=1\nmissing_pl=pd.DataFrame(all_pl, index = missing_ids)"

In [3]:
missing_pl

,Id,Classe,Eta',Nome Completo,Nazionalita'
2097,2097,28/02/2000,21,Kean Moise,"Italia, Costa d'Avorio"
4184,4184,23/06/1998,23,Brekalo Josip,Croazia
2065,2065,14/03/1994,27,Praet Dennis,Belgio
5694,5694,31/01/1998,23,Beto,"Portogallo, Guinea Bissau"
5685,5685,28/02/1998,23,Koopmeiners Teun,Olanda
5675,5675,25/05/2000,21,Theate Arthur,Belgio
5677,5677,24/06/2000,21,Perez Nehuen,Argentina
2614,2614,14/12/1995,26,Odriozola Alvaro,Spagna
4719,4719,14/09/2000,21,Ampadu Ethan,"Galles, Inghilterra"
4220,4220,16/11/1995,26,Zambo Anguissa André,Camerun


In [4]:
ST_INFO.rename(columns = {'Qt. A': 'Qt_A','Qt. I': 'Qt_I'}, inplace=True)
ST_INFO=ST_INFO.drop(columns='Diff.')
ST_INFO.to_pickle("Statistiche_giocatori.pkl")
missing_pl.to_pickle('Missing_Players.pkl')
ST_INFO

,Id,R,Nome,Squadra,Pg,Mv,Mf,Gf,Gs,Rp,...,R+,R-,Ass,Amm,Esp,Au,Qt_A,Qt_I,Nome Squadra,Allenatore
785,785,A,IMMOBILE,Lazio,2,7.50,12.00,3,0,0,...,1,1,0,0,0,0,40,38,PALLA PAZZA,nanni
507,507,A,MURIEL,Atalanta,2,6.25,7.75,1,0,0,...,0,0,0,0,0,0,39,39,PDG 1908,pietro
2841,2841,A,VLAHOVIC,Fiorentina,2,6.25,7.50,1,0,0,...,0,0,0,1,0,0,34,34,SOROS FC,musci8
608,608,A,ZAPATA D.,Atalanta,0,0.00,0.00,0,0,0,...,0,0,0,0,0,0,32,33,PDG 1908,pietro
2764,2764,A,MARTINEZ L.,Inter,1,7.00,9.50,1,0,0,...,0,0,0,1,0,0,32,32,MAINZ NA GIOIA,franky
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5477,5477,A,WARMING,Torino,0,0.00,0.00,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None
5493,5493,A,BOCALON,Venezia,0,0.00,0.00,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None
5360,5360,A,VERGANI,Salernitana,0,0.00,0.00,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None
4483,4483,A,CANGIANO,Bologna,0,0.00,0.00,0,0,0,...,0,0,0,0,0,0,1,1,svincolato,None


In [5]:
df=pd.read_pickle('Missing_Players.pkl')
df

,Id,Classe,Eta',Nome Completo,Nazionalita'
2097,2097,28/02/2000,21,Kean Moise,"Italia, Costa d'Avorio"
4184,4184,23/06/1998,23,Brekalo Josip,Croazia
2065,2065,14/03/1994,27,Praet Dennis,Belgio
5694,5694,31/01/1998,23,Beto,"Portogallo, Guinea Bissau"
5685,5685,28/02/1998,23,Koopmeiners Teun,Olanda
5675,5675,25/05/2000,21,Theate Arthur,Belgio
5677,5677,24/06/2000,21,Perez Nehuen,Argentina
2614,2614,14/12/1995,26,Odriozola Alvaro,Spagna
4719,4719,14/09/2000,21,Ampadu Ethan,"Galles, Inghilterra"
4220,4220,16/11/1995,26,Zambo Anguissa André,Camerun
